# 人脸识别
### 用Inception网络搭建的人脸识别框架，由于图片通道放在了第一维度，需要用tensorflow-gpu跑哦

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "2"
import tensorflow as tf
import cv2
import fr_utils as u

import tensorflow.keras.backend as K
#K.set_image_data_format('channels_first')

In [ ]:
def img_to_encoding(image_path, model):
    img1 = cv2.imread(image_path, 1)
    img = img1[...,::-1]
    img = tf.transpose(img, (2,0,1))/255    # 此处img类型为tf.int8，如果除255.0会因为类型不同报错
    x_train = tf.convert_to_tensor(img)[tf.newaxis,...]     # 使用tf.newaxis为数组增加一个维度
    print(x_train.shape)
    #re_train=tf.transpose(x_train,perm=[0,2,3,1])
    #print(re_train.shape)
    embedding = model.predict_on_batch(x_train)
    return embedding


In [ ]:
def creat_database(FRmodel):
    """
    创建面部识别的数据库
    Returns:
        database: 包含姓名和对应面部编码的字典
    """
    database = {}
    database["danielle"] = img_to_encoding("./images/danielle.png", FRmodel)
    database["younes"] = img_to_encoding("./images/younes.jpg", FRmodel)
    database["tian"] = img_to_encoding("./images/tian.jpg", FRmodel)
    database["andrew"] = img_to_encoding("./images/andrew.jpg", FRmodel)
    database["kian"] = img_to_encoding("./images/kian.jpg", FRmodel)
    database["dan"] = img_to_encoding("./images/dan.jpg", FRmodel)
    database["sebastiano"] = img_to_encoding("./images/sebastiano.jpg", FRmodel)
    database["bertrand"] = img_to_encoding("./images/bertrand.jpg", FRmodel)
    database["kevin"] = img_to_encoding("./images/kevin.jpg", FRmodel)
    database["felix"] = img_to_encoding("./images/felix.jpg", FRmodel)
    database["benoit"] = img_to_encoding("./images/benoit.jpg", FRmodel)
    database["arnaud"] = img_to_encoding("./images/arnaud.jpg", FRmodel)

    return database


In [ ]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    根据公式（4）实现三元组损失函数
    
    参数：
        y_true -- true标签，当你在Keras里定义了一个损失函数的时候需要它，但是这里不需要。
        y_pred -- 列表类型，包含了如下参数：
            anchor -- 给定的“anchor”图像的编码，维度为(None,128)
            positive -- “positive”图像的编码，维度为(None,128)
            negative -- “negative”图像的编码，维度为(None,128)
        alpha -- 超参数，阈值
    
    返回：
        loss -- 实数，损失的值
    """
    #获取anchor, positive, negative的图像编码
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    #第一步：计算"anchor" 与 "positive"之间编码的距离，这里需要使用axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis=-1)
    
    #第二步：计算"anchor" 与 "negative"之间编码的距离，这里需要使用axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1)
    
    #第三步：减去之前的两个距离，然后加上alpha
    basic_loss = tf.add(tf.subtract(pos_dist,neg_dist),alpha)
    
    #通过取带零的最大值和对训练样本的求和来计算整个公式
    loss = tf.reduce_sum(tf.maximum(basic_loss,0))
    
    return loss


In [ ]:
def verify(image_path, identity, database, model):
    """
    人脸认证，比较摄像头图像与id信息是否符合，即比较库中identity的编码和image_path的编码（即全连接层的输出）
    Args:
        image_path: 摄像头的图片
        identity: 字符串，想要验证的人的名字
        database: 字典， 包含了成员姓名和对应编码
        model: 训练好的模型
    Returns:
        dist: 摄像头中图片与数据库中图片的编码差距
        is_open_door： True/False 是否开门
    """
    # 计算图像的编码
    # 计算图像的编码，使用fr_utils.img_to_encoding()来计算。
    encoding = img_to_encoding(image_path, model)
    # 计算与数据库中保存的编码的差距，这里使用tf的范数计算函数替换原文的np.linalg.norm
    dist = tf.norm(encoding - database[identity])
    # 判断是否打开门
    if dist < 0.7:
        print("欢迎 " + str(identity) + "回家！")
        is_door_open = True
    else:
        print("经验证，您与" + str(identity) + "不符！")
        is_door_open = False

    return dist, is_door_open

In [ ]:
def who_is_it(image_path, database, model):
    """
    根据指定的图片来进行人脸识别
    Args:
        image_path: 图片地址
        database: 包含了名字与比那吗的字典
        model: 训练好的图形
    Returns:
        min_dist: 字典中与输入图像最相近的编码
        identity: 与min_dist对应的名字
    """
    # 计算指定图像的编码，使用fr_utils.img_to_encoding()来计算。
    encoding = img_to_encoding(image_path, model)
    # 找到最相近的编码
    min_dist = 100                  # 初始化min_dist变量为足够大的数字，这里设置为100
    # 遍历数据库，找到min_dist
    for name, db_enc in database.items():
        dist = tf.norm(encoding - db_enc)
        if dist < min_dist:
            min_dist = dist
            identity = name
    # 判断输入图像是否在数据库中
    if min_dist > 0.7:
        print("抱歉，您的信息不在数据库中。")
    else:
        print("姓名" + identity + "  差距：" + str(min_dist))

    return min_dist, name


In [ ]:
FRmodel = u.faceRecoModel(input_shape=(3, 96, 96))
# 编译模型
FRmodel.compile(optimizer='adam', loss=triplet_loss, metrics=['accuracy'])
# 加载权值
u.load_weights_from_FaceNet(FRmodel)


In [ ]:
# 加载数据库
database = creat_database(FRmodel)

In [ ]:
who_is_it("./images/camera_0.jpg", database, FRmodel)